In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from time import sleep

In [2]:
import re

In [3]:
from time import sleep

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
import pandas as pd

In [6]:
s = Service("C:\geckodriver.exe")
driver = webdriver.Firefox(service= s)
wait = WebDriverWait(driver, 15)

C:\Users\owlsense.intern\AppData\Local\Temp\ipykernel_3440\135083102.py:1: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  s = Service("C:\geckodriver.exe")


In [7]:
# Patterns re
pattern_for_production = r'\$([\d,]+)'
pattern_for_gross = r'\d+'


In [27]:
# LISTS
Date = []
Gross = []
Treaters = []
Days = []
Rank = []  
Name = []   
Genre = []  
Production_Budget = []
MPAA = []

In [46]:
URL = '''https://www.the-numbers.com/market/2022/top-grossing-movies'''

In [47]:
driver.get(URL)

In [48]:
html = driver.page_source

In [49]:
soup = BeautifulSoup(html, 'lxml')

In [50]:
tbody = soup.find('tbody')

tr_elements_1 = tbody.find_all('tr')

sleep(2)

for movie_list_tr in tr_elements_1[134:-2]:
    #______________
    pro_match_ = ''
    MPAA_VALUE = ''
    #____________
    td_elements_1 = movie_list_tr.find_all('td')
    
    # 1st td with class='data'
    movie_rank = td_elements_1[0].text
    #Rank.append(movie_rank)
    
    # 2nd td with no class (text and href)
    movie_name_href = td_elements_1[1]
    movie_name = movie_name_href.text
    #Name.append(movie_name)
    second_td_href = movie_name_href.find('a')['href'] if movie_name_href.find('a') else None
    # 5th td
    movie_genre = td_elements_1[4].text
    #Genre.append(movie_genre)
    
    # Click on the link in the 2nd td if it exists
    if second_td_href:
        driver.get("https://www.the-numbers.com"+second_td_href)
        wait.until(presence_of_element_located((By.XPATH,'''//*[@id="page_filling_chart"]''')))
        sleep(1)
        # Perform your desired actions on the new page
                                    #   2
        html_2= driver.page_source

        soup_2 = BeautifulSoup(html_2, 'lxml')

        # Find the div with id 'page_filling_chart'
        div_page_filling_chart = soup_2.find('div', id = 'summary', class_='content active')

        # Find all tables within the div
        tables = div_page_filling_chart.find_all('table')

        #--------------------------------------------Production-budget--------------------------------
        # Check if there are at least two tables
        if len(tables) >= 2:
            # Get the 2nd table
            table_4_pro = tables[1]

            # Find the tbody within the 2nd table
            pro_tbody = table_4_pro.find('tbody')

            # Find the 4th tr within the tbody
            pro_tr_elements = pro_tbody.find_all('tr')

            if len(pro_tr_elements) >= 4:
                pro_fourth_tr = pro_tr_elements[3]  # 4th tr (index 3)

                # Find the 2nd td within the 4th tr and extract its text
                pro_second_td = pro_fourth_tr.find_all('td')[1]  # 2nd td (index 1)
                pro_result = pro_second_td.get_text()

                #apply regex pattern_for_production
                pro_match = re.search(pattern_for_production, pro_result)

                if pro_match:
                    pro_match = pro_match.group(1)
                    # Remove commas from the number and convert it to an integer
                    pro_match_ = int(pro_match.replace(',', ''))
                    #Production_Budget.append(production_bud)
                else:
                    print("No match found.")

            else:
                print("There are not enough rows in the tbody.")
        else:
            print("There are not enough tables within the div.")

        #---------------------------------------------------MPAAA----------------------------------------------------

        # Check if there are at least three tables
            
        if len(tables) >= 5:
            # Get the 4th table
            MPA_table = tables[4]
            
            # Find the tbody within the 3rd table
            MPA_tbody = MPA_table.find('tbody')
            # Find the 4th tr within the tbody
            MPA_tr_elements = MPA_tbody.find_all('tr')
            if len(MPA_tr_elements) >= 4:
                MPA_fourth_tr = MPA_tr_elements[3]  # 4th tr (index 3)
                # Find the 2nd td within the 4th tr
                MPA_second_td = MPA_fourth_tr.find_all('td')[1]  # 2nd td (index 1)
                # Find the 'a' tag within the 2nd td and extract its text
                MPA_a_tag = MPA_second_td.find('a')
                if MPA_a_tag:
                    MPAA_VALUE = MPA_a_tag.text
                    # Print or use 'result' as needed
                    #MPAA.append(MPAA_VALUE)
                else:
                    print("No 'a' tag found within the 2nd td.")
                        
        elif len(tables) >= 4:
            # Get the 3rd table
            MPA_table = tables[3]
            
            # Find the tbody within the 3rd table
            MPA_tbody = MPA_table.find('tbody')
            # Find the 4th tr within the tbody
            MPA_tr_elements = MPA_tbody.find_all('tr')
            if len(MPA_tr_elements) >= 4:
                MPA_fourth_tr = MPA_tr_elements[3]  # 4th tr (index 3)
                # Find the 2nd td within the 4th tr
                MPA_second_td = MPA_fourth_tr.find_all('td')[1]  # 2nd td (index 1)
                # Find the 'a' tag within the 2nd td and extract its text
                MPA_a_tag = MPA_second_td.find('a')
                if MPA_a_tag:
                    MPAA_VALUE = MPA_a_tag.text
                    # Print or use 'result' as needed
                    #MPAA.append(MPAA_VALUE)
                else:
                    print("No 'a' tag found within the 2nd td.")
#                     MPAA_VALUE = ''
                
            
            else:
                print("There are not enough rows in the tbody.")
#                 MPAA_VALUE = ''
        else:
            print("There are not enough tables within the div.")
            
        
        
        #After Production and MPAA, with same driver click on 'BOX OFFICE BUTTON' and scrape
        Box_Office_BUtton = driver.find_element(By.CSS_SELECTOR, '''#a_box_office''')
        #By.XPATH, '''/html/body/div/div[3]/div[3]/ol/li[3]'''
        if Box_Office_BUtton is not None:
            Box_Office_BUtton.click()
        
        
        wait.until(presence_of_element_located((By.XPATH,'''//*[@id="box_office_chart"]''')))
        sleep(1)
        
        html_3 = driver.page_source
        soup_3 = BeautifulSoup(html_3, 'lxml')
        main_divs = soup_3.find('div', id='box-office', class_='content active')
        all_divs = main_divs.find_all('div', id='box_office_chart')
        try:
            box_office_box_table = all_divs[1].find('table')
        except:
            box_office_box_table = all_divs[0].find('table')
        # Find all the 'tr' elements within the table
        box_office_tr_elements = box_office_box_table.find_all('tr')

        # Loop through each 'tr' element
        for box_office_tr in box_office_tr_elements[1:]:
            box_office_td_elements = box_office_tr.find_all('td')
            # Check if there are enough 'td' elements in the row
            if len(box_office_td_elements) >= 9:
                
                #append movie rank
                Rank.append(int(movie_rank))
                #append movie name
                Name.append(movie_name)
                #append movie genre
                Genre.append(movie_genre)
                #append movie production budget
                Production_Budget.append(pro_match_)
                #append movie MPAA
                MPAA.append(MPAA_VALUE)
                
                # 1st td (index 0) - Date
                Date.append(box_office_td_elements[0].text.strip())

                # 3rd td (index 2) - Gross
                Gross.append(int(''.join(re.findall(pattern_for_gross, box_office_td_elements[2].text.strip()))))

                # 6th td (index 5) - Treaters
                Treaters.append(int(box_office_td_elements[5].text.strip().replace(',','')))

                # 9th td (index 8) - Days
                Days.append(box_office_td_elements[8].text.strip())
                
    print(f"Scraping movie at rank: {movie_rank} Completed!!!!!")    
    # Go back to the previous page
    driver.back()
    wait.until(presence_of_element_located((By.XPATH,'''//*[@id="page_filling_chart"]''')))
    sleep(1)
    driver.back()
    wait.until(presence_of_element_located((By.XPATH,'''/html/body/div/div[3]/div[1]/table''')))
    sleep(1)
    
print("ALL DONE 🎉🎉🎉")

There are not enough rows in the tbody.
Scraping movie at rank: 134 Completed!!!!!
No match found.
Scraping movie at rank: 135 Completed!!!!!
There are not enough rows in the tbody.
Scraping movie at rank: 136 Completed!!!!!
No match found.
No 'a' tag found within the 2nd td.
Scraping movie at rank: 137 Completed!!!!!
No match found.
Scraping movie at rank: 138 Completed!!!!!
No match found.
Scraping movie at rank: 139 Completed!!!!!
Scraping movie at rank: 140 Completed!!!!!
No match found.
Scraping movie at rank: 141 Completed!!!!!
No match found.
Scraping movie at rank: 142 Completed!!!!!
Scraping movie at rank: 143 Completed!!!!!
No match found.
No 'a' tag found within the 2nd td.
Scraping movie at rank: 144 Completed!!!!!
No match found.
Scraping movie at rank: 145 Completed!!!!!
No match found.
Scraping movie at rank: 146 Completed!!!!!
No match found.
No 'a' tag found within the 2nd td.
Scraping movie at rank: 147 Completed!!!!!
No match found.
Scraping movie at rank: 148 Comple

Scraping movie at rank: 251 Completed!!!!!
No match found.
No 'a' tag found within the 2nd td.
Scraping movie at rank: 252 Completed!!!!!
No match found.
Scraping movie at rank: 253 Completed!!!!!
No match found.
No 'a' tag found within the 2nd td.
Scraping movie at rank: 254 Completed!!!!!
No match found.
Scraping movie at rank: 255 Completed!!!!!
No match found.
Scraping movie at rank: 256 Completed!!!!!
No match found.
Scraping movie at rank: 257 Completed!!!!!
No match found.
No 'a' tag found within the 2nd td.
Scraping movie at rank: 258 Completed!!!!!
No match found.
No 'a' tag found within the 2nd td.
Scraping movie at rank: 259 Completed!!!!!
No match found.
Scraping movie at rank: 260 Completed!!!!!
There are not enough rows in the tbody.
Scraping movie at rank: 261 Completed!!!!!
No match found.
No 'a' tag found within the 2nd td.
Scraping movie at rank: 262 Completed!!!!!
There are not enough rows in the tbody.
Scraping movie at rank: 263 Completed!!!!!
There are not enough 

TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:188:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:506:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16


In [56]:
df = pd.DataFrame({"Rank":Rank, "Name":Name, "Genre":Genre, "Production_Budget":Production_Budget, "MPAA":MPAA, "Date":Date,"Gross":Gross,  "Treaters":Treaters, "Days":Days })

In [59]:
df.sample(10)

,Rank,Name,Genre,Production_Budget,MPAA,Date,Gross,Treaters,Days
6842,253,The French Dispatch,Comedy,,R,"Oct 28, 2021",109182,52,7
869,75,West Side Story,Musical,100000000,PG-13,"Feb 17, 2022",29725,450,70
2807,111,Moonage Daydream,Documentary,,PG-13,"Oct 11, 2022",24699,290,26
476,70,X,Horror,1000000,R,"Apr 30, 2022",8215,49,44
5808,210,Waterman,Documentary,,Create your own comparison chart…,"Apr 21, 2022",5519,42,21
2824,111,Moonage Daydream,Documentary,,PG-13,"Oct 28, 2022",8338,52,43
3053,114,Family Camp,Comedy,,,"May 22, 2022",254008,1057,10
3407,121,Breaking,Thriller/Suspense,,,"Aug 31, 2022",89265,902,6
1173,81,Redeeming Love,Drama,,PG-13,"Jan 21, 2022",1493775,1903,1
5463,195,King Richard,Drama,50000000,PG-13,"Feb 19, 2022",7844,151,93


In [61]:
df.to_csv("Movies_Detail_from_61_till_rank_269.csv",index=False)

In [60]:
df.tail()

,Rank,Name,Genre,Production_Budget,MPAA,Date,Gross,Treaters,Days
7187,269,Pleasure,Drama,,R,"May 29, 2022",4788,33,17
7188,269,Pleasure,Drama,,R,"May 30, 2022",4714,33,18
7189,269,Pleasure,Drama,,R,"May 31, 2022",3033,33,19
7190,269,Pleasure,Drama,,R,"Jun 1, 2022",3918,33,20
7191,269,Pleasure,Drama,,R,"Jun 2, 2022",3485,33,21
